## Serial LETK Filtering


In [ ]:
import Simulator
import Observation
import Statistics
import SLETKalmanFilter

## Loading simulator 

For Kalman filtering we have to be able to propagate a Gaussian distribution in time and to have access to the linear operators

In [ ]:
timestamp = "2022_03_02-12_44_46"

grid, simulator = Simulator.from_file(timestamp)

## Definition of initial distribution 

The inital state is equipped with a Gaussian distribution specified by its mean and covariance. 

In [ ]:
statistics = Statistics.Statistics(simulator, N_e=50, safe_history=True)

In [ ]:
prior_args = Statistics.prior_args_from_file(timestamp)
statistics.set_prior(prior_args)

statistics.plot()

## Loading observations 

Observation positions which are fixed for a setup and values from the truth

In [ ]:
obs_timestamp = "2022_03_02-12_44_53"
observation = Observation.from_file(grid, timestamp, obs_timestamp)

In [ ]:
observation.plot_positions()

## LETKF filtering

Updating/conditioning the distributions at every observation time

In [ ]:
# We choose the scale_r as about 2/3 of the correlation length in grid cells 
scale_r = 6

In [ ]:
sletkFilter = SLETKalmanFilter.SLETKalman(statistics, observation, scale_r)

In [ ]:
sletkFilter.groups

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(sletkFilter.W_loc, origin="lower", vmin=0.0, vmax=1.0)
plt.colorbar()
plt.show()
for g in range(len(sletkFilter.groups)):
    plt.imshow(sletkFilter.W_analyses[g], origin="lower", vmin=0.0, vmax=1.0)
    plt.colorbar()
    plt.show()

In [ ]:
for t in range(observation.N_obs):
    print(t)
    statistics.propagate(25)
    sletkFilter.filter(statistics.ensemble.ensemble, observation.obses[t])
    statistics.plot()